In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [2]:
trash = pd.read_csv('../data/trash_hauler_report.csv')

In [3]:
trash['Zip Code'] = trash['Zip Code'].astype(str)
trash['Council District'] = trash['Council District'].astype(str)
trash['Zip Code'] = trash['Zip Code'].str[:5]
trash['Council District'] = trash['Council District'].str[:-2]
trash['Incident Address'] = trash['Incident Address'].str.title()
trash.columns = trash.columns.str.rstrip()
trash['Incident Address'] = trash['Incident Address'].str.split(',', expand = True)[0]
trash['Description'] = trash['Description'].str.title() 
trash['miss_in_description'] = trash['Description'].str.extract(r'(Miss)')
trash['Trash Hauler'] = trash['Trash Hauler'].str.title()

In [4]:
red_river = trash.loc[(trash['Trash Hauler'] == 'Red River')]
red_river_misses = red_river.loc[(red_river['Request'] == 'Trash - Curbside/Alley Missed Pickup') 
                                |(red_river['miss_in_description'] == 'Miss')]
red_river_misses.shape

(12838, 12)

In [5]:
red_river_fines = ((red_river_misses['Incident Address'].value_counts()-1)*200).to_frame().reset_index()
red_river_fines = red_river_fines.rename(columns = {'count':'Fine'})

In [6]:
red_river_fines['Fine'].sum()

919400

In [60]:
trash_misses = trash.loc[(trash['Request'] == 'Trash - Curbside/Alley Missed Pickup') 
                        |(trash['miss_in_description'] == 'Miss')]

In [8]:
trash_misses['Trash Route'].value_counts().to_frame().reset_index()

,Trash Route,count
0,4504,329
1,3302,291
2,4404,255
3,9303,245
4,1303,240
...,...,...
167,4504S,3
168,3303S,2
169,2505S,2
170,4201S,2


In [9]:
total_fines = ((trash_misses['Incident Address'].value_counts()-1)*200).to_frame().reset_index()
total_fines = total_fines.rename(columns = {'count':'Fine'})

In [62]:
total_misses_by_zip = trash_misses.groupby('Zip Code')['Trash Hauler'].value_counts().reset_index()

In [11]:
miss = trash_misses['Trash Hauler'].value_counts().to_frame().reset_index()
miss

,Trash Hauler,count
0,Red River,12838
1,Metro,3022
2,Waste Ind,1128


In [12]:
total = trash['Trash Hauler'].value_counts().to_frame().reset_index()
total

,Trash Hauler,count
0,Red River,14395
1,Metro,3580
2,Waste Ind,1350


In [13]:
compare = pd.merge(miss, total, left_on = 'Trash Hauler', right_on = 'Trash Hauler')
compare = compare.rename(columns = {'count_x':'missed', 'count_y':'total'})

In [14]:
compare['Pct Missed'] = compare['missed']/compare['total']
compare

,Trash Hauler,missed,total,Pct Missed
0,Red River,12838,14395,0.891837
1,Metro,3022,3580,0.844134
2,Waste Ind,1128,1350,0.835556


In [38]:
trash.groupby('Trash Hauler')['Trash Route'].value_counts()

Trash Hauler  Trash Route
Metro         9303           267
              9203           241
              9208           232
              9503           195
              9403           168
                            ... 
Waste Ind     4407            71
              2408            46
              2306            44
              4508            39
              2207            24
Name: count, Length: 177, dtype: int64

In [52]:
trash_misses.groupby('Trash Hauler')['Trash Route'].nunique().to_frame().reset_index()

,Trash Hauler,Trash Route
0,Metro,33
1,Red River,125
2,Waste Ind,16


In [ ]:
all_hauls_missed = trash.loc[(trash['Request'] == 'Trash - Curbside/Alley Missed Pickup') | (trash['Miss_In_Description'] == 'Miss')]
hauler_perf = all_hauls_missed.groupby('Trash Hauler')['Request Number'].count().to_frame().reset_index()
hauler_routes = trash.groupby('Trash Hauler')['Trash Route'].nunique().to_frame().reset_index()
hauler_routes
hauler_merged= pd.merge(hauler_routes, hauler_perf, on = 'Trash Hauler', how = 'inner')
hauler_merged.columns = ['Hauler','Number of Routes','Number of Miss Complaints']
hauler_merged['Avg Misses per Route'] = (hauler_merged['Number of Miss Complaints'] / hauler_merged['Number of Routes']).round(2)
hauler_merged
sns.barplot(x = 'Hauler', y = 'Avg Misses per Route', data = hauler_merged, hue = 'Hauler', palette = 'Dark2')
plt.title('Missed Pickup Complaints per Assigned Route');

In [64]:
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,miss_in_description
0,25270,11/01/17,Trash - Backdoor,"House With The Wheel Chair Ramp, They Share Dr...",3817 Crouch Dr,37207,Red River,3205,2,1.727970e+06,686779.478089,NaN
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash Miss Tuesday.,4028 Clarksville Pike,37218,Red River,4202,1,1.721259e+06,685444.799565,Miss
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash Miss Tuesday.,6528 Thunderbird Dr,37209,Red River,4205,20,1.707027e+06,659887.471571,Miss
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed,2603 Old Matthews Rd,37207,Waste Ind,2206,2,1.735692e+06,685027.245923,Miss
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed The Even Side Of The Road.,604 Croley Dr,37209,Red River,4203,20,1.710186e+06,664205.101066,Miss


In [74]:
pd.to_datetime(trash['Date Opened'],format='%m/%d/%y').dt.strftime('%Y-%m-%d')

0        2017-11-01
1        2017-11-01
2        2017-11-01
3        2017-11-01
4        2017-11-01
            ...    
20221    2019-11-01
20222    2019-11-01
20223    2019-11-01
20224    2019-11-01
20225    2019-11-01
Name: Date Opened, Length: 20226, dtype: object